## Generación de base de datos de palabras en Español

In [1]:
# librerias utilizadas
import pandas as pd
import nltk
import string
import datetime
from collections import Counter
nltk.download("punkt")
nltk.download("stopwords")


[nltk_data] Downloading package punkt to /home/jogabell/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jogabell/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [74]:
# Lectura de base de datos
gh_2015=pd.read_excel('data/DATA_PLATAFORMA_HIPGIVE.xlsx',sheet_name='2015',encode='latin1')
gh_2016=pd.read_excel('data/DATA_PLATAFORMA_HIPGIVE.xlsx',sheet_name='2016',encode='latin1')
gh_2017=pd.read_excel('data/DATA_PLATAFORMA_HIPGIVE.xlsx',sheet_name='2017',encode='latin1')
gh_2018=pd.read_excel('data/DATA_PLATAFORMA_HIPGIVE.xlsx',sheet_name='2018',encode='latin1')
gh_2019=pd.read_excel('data/DATA_PLATAFORMA_HIPGIVE.xlsx',sheet_name='2019',encode='latin1') 

In [75]:
gh_2019.columns

Index([u'Project ID', u'Project Date', u'Name', u'Organization',
       u'EIN or Tax ID', u'IRS 501 or Equivalent', u'Goal', u'Total',
       u'Percentage Raised', u'HIP Give Fee', u'HIP Give Percent',
       u'HIP Give Tip', u'WePay Fee', u'Net Amount',
       u'Number of Individual Donors', u'Matching Funds Total', u'Start Date',
       u'End Date', u'Name of Main Contact', u'Main Contact Email',
       u'Main Contact Phone Number', u'OwnerID', u'Address', u'State', u'City',
       u'ZIP', u'Category', u'Category2', u'Short Desc EN', u'Short DescSP',
       u'Long Desc EN', u'Long Desc SP', u'Goal EN', u'Goal SP', u'Measure EN',
       u'Measure SP', u'Problem', u'Budget', u'Mission EN', u'Mission SP',
       u'About EN', u'About SP', u'Pop Served', u'Published Status',
       u'Date Approved', u'Twitter', u'YouTube', u'Author of Post',
       u'Author Email', u'Board Members List', u'Incorporation as Nonprofit',
       u'IRS Determination Letter', u'IRS 990 or Audited Financial Stat

In [76]:
features=[u'Project ID', u'Name', u'Organization',u'Published Status',
       u'Goal', u'Percentage Raised', u'Number of Individual Donors',  
       u'Short DescSP', u'Long Desc SP',
       u'Short Desc EN',u'Long Desc EN', ]
_2015=gh_2015[features]
_2016=gh_2016[features]
_2017=gh_2017[features]
_2018=gh_2018[features]
_2019=gh_2019[features]

In [77]:
_2015['Published Status'].value_counts()

ended       175
draft        87
private       3
rejected      1
Name: Published Status, dtype: int64

In [78]:
# Limpiado base de datos. Se quitando registros con 'Published Status'='draft' o 'rejected'

data=_2015.append(_2016).append(_2017).append(_2018).append(_2019)

In [79]:
data.shape

(3966, 11)

In [80]:
mask1=(data['Published Status']!= 'draft')

In [81]:
data['Published Status'].value_counts()

draft        3105
ended         749
rejected       70
publish        28
private        10
submitted       2
archived        1
Name: Published Status, dtype: int64

In [82]:
data=data[mask1]

In [83]:
data['Published Status'].value_counts()

ended        749
rejected      70
publish       28
private       10
submitted      2
archived       1
Name: Published Status, dtype: int64

In [84]:
mask2=(data['Published Status']!= 'rejected')

In [85]:
data=data[mask2]

In [86]:
data['Published Status'].value_counts()

ended        749
publish       28
private       10
submitted      2
archived       1
Name: Published Status, dtype: int64

## Creación de base de datos de texto para 3 categorías de recaudación

#### Base con una recaudación menos al 35%

In [87]:
data_less_than_35 = data[data[u'Percentage Raised']<=35]

#### Base con una recaudación entre 36% y el 70% 

In [88]:
# selecting rows based on condition 
data_between_36_and_70= data[(data[u'Percentage Raised']>35) & data[u'Percentage Raised']<=70]

#### Base con una recaudación igual o mayor a 71%

In [89]:
data_grater_than_70=data[data[u'Percentage Raised']>70]

## Función para preparar los datos de texto

In [133]:
def text_analysis(serie):
    s=serie.isna().value_counts()    
    if s.size==2:
          serie=serie.dropna()
    elif (s.size==1)&(s[0]==True):
      print('no existen datos')
    datetime_object = datetime.datetime.now()
    lista=[]    
    for u_text in serie:
        if(type(u_text)!= type(datetime_object)):            
            lista.append(u_text.encode("utf-8"))
    text_string_type=', '.join(lista)
    text_string_type=unicode(text_string_type, "utf-8").lower()    
    
    useless_words=nltk.corpus.stopwords.words("spanish")+nltk.corpus.stopwords.words("english")+list(string.punctuation)+[u'cada',u'través',u'para',u'\xa1que',u'\xa1to',u'son',u'con',u'la',u'se',u'en',u'el',u'que',u'la',u'las',u'lo',u'los',u'de',u'the',u'us',u'we',u'A',u'nbsp',u'8217',u'8211',u'8220',u'8221',u'\u2019',u'\u201d',u'\u201c',u'pa\xeds',u'visi\xf3n',u'ray\xf3n',u'pa\xeds',u'tecnolog\xeda',u'ay\xfadanos',u'\xa1que',u'harder\u2014but',u'michoac\xe1n',u'let\xb4s',u'\xa1to',u'granny\xb4s',u'espa\xf1ol.',u'.']#+list(u"\u2019")+list('...')    
    words=nltk.word_tokenize(text_string_type)
    words_filtered=[word for word in words if not word in useless_words]
    
    words_counter=Counter(words_filtered)
    most_common_words = words_counter.most_common()
    return most_common_words

In [180]:
def text_analysis_id_proyect_words(serie):
    project_ids=serie['Project ID'].values
    ShortDescSP=serie[u'Short DescSP'].values
    LongDescSP=serie[u'Long Desc SP'].values
    ShortDescEN=serie[u'Short Desc EN'].values
    LongDescEN=serie[u'Long Desc EN'].values
    
    data_list=[]
    for i in range(len(project_ids)):
        data_list.append([project_ids[0],[ShortDescSP[0],LongDescSP[0],ShortDescEN[0],LongDescEN[0]]])
    
    shortDescSP_lnew = []
    longDescSP_lnew  = []
    shortDescEN_lnew = []
    longDescEN_lnew  = []
    
    new_data_list=[]
    for e in data_list:
        shortDescSP = e[1][0]
        longDescSP  = e[1][1]
        shortDescEN = e[1][2]
        longDescEN  = e[1][3]
        
        desc_list=[]
        if((type(shortDescSP)!= type(datetime_object)) &(type(longDescSP)!= type(datetime_object)) & (type(shortDescEN)!= type(datetime_object)) & (type(longDescEN)!= type(datetime_object))    ):
            new_data_list.append(e[0],[shortDescSP,longDescSP,shortDescEN,longDescEN])
        else:
            new_data_list.append(e[0],[])
            
            shortDescSP_lnew.append(shortDescSP.encode("utf-8"))
    
    text_string_type=unicode(text_string_type, "utf-8").lower()    
    
    useless_words=nltk.corpus.stopwords.words("spanish")+nltk.corpus.stopwords.words("english")+list(string.punctuation)+[u'cada',u'través',u'para',u'\xa1que',u'\xa1to',u'son',u'con',u'la',u'se',u'en',u'el',u'que',u'la',u'las',u'lo',u'los',u'de',u'the',u'us',u'we',u'A',u'nbsp',u'8217',u'8211',u'8220',u'8221',u'\u2019',u'\u201d',u'\u201c',u'pa\xeds',u'visi\xf3n',u'ray\xf3n',u'pa\xeds',u'tecnolog\xeda',u'ay\xfadanos',u'\xa1que',u'harder\u2014but',u'michoac\xe1n',u'let\xb4s',u'\xa1to',u'granny\xb4s',u'espa\xf1ol.',u'.']#+list(u"\u2019")+list('...')    
    words=nltk.word_tokenize(text_string_type)
    words_filtered=[word for word in words if not word in useless_words]


    return lista

In [181]:
hola=text_analysis_id_proyect_words(data_less_than_35)

[[10057, [u'Prevenci\xf3n de la Violencia a trav\xe9s de actividades musicales y culturales. \xa1Se parte de nuestros esfuerzos por un mejor El Salvador!', u'\xbfEst\xe1s listo para cambiar la realidad de ni\xf1os y j\xf3venes que viven en violencia y riesgo social? \xa1\xa1\xa1Se parte de un movimiento musical que cambia el llanto de la violencia por el sonido de la m\xfasica!!! Nuestro objetivo es mejorar la calidad de vida de la ni\xf1ez y juventud destinataria, a trav\xe9s de la promoci\xf3n de la educaci\xf3n musical gratuita, inclusi\xf3n social y participaci\xf3n en actividades culturales y musicales como alternativa a la violencia en El Salvador. Ante una problem\xe1tica compleja de un pa\xeds convulsionado lleno de violencia y falta de oportunidades que generan formaci\xf3n de grupos delinquiles, es necesario dar una respuesta compleja. Por tal motivo, en el 2012 la Fundaci\xf3n EDYTRA con apoyo del Banco Mundial, fund\xf3 la Orquesta y Coro Sinf\xf3nico Juvenil del Pol\xedgon

In [161]:
len(data_less_than_35['Project ID'].values)

537

In [170]:
data_less_than_35['Short Desc EN'].values[0]

u'Prevention of Violence through musical and cultural activities. Let\xb4s play music for a peaceful El Salvador!'

## Obtención de Short Desciption en Español para 3 categorías

In [91]:
# Short Description Español Categoría 1 (recaudación<35%)
ShortDescESP_Cat1=data_less_than_35['Short DescSP']

In [92]:
# Short Description Español Categoría 2 (35<recaudación<=70%)
ShortDescESP_Cat2=data_between_36_and_70['Short DescSP']

In [93]:
# Short Description Español Categoría 3 (70<recaudación)
ShortDescESP_Cat3=data_grater_than_70['Short DescSP']

#### Quitando registros sin descripción

In [94]:
ShortDescESP_Cat1=ShortDescESP_Cat1.dropna()
ShortDescESP_Cat2=ShortDescESP_Cat2.dropna()
ShortDescESP_Cat3=ShortDescESP_Cat3.dropna()

In [95]:
ShortDescESP_Cat1.shape

(492,)

In [96]:
ShortDescESP_Cat2.shape

(736,)

In [97]:
ShortDescESP_Cat3.shape

(159,)

# Análisis

de texto para cada categoría

In [98]:
# Obtención de las palabras más utilizadas categoría 1
mcw_Cat1 = text_analysis(ShortDescESP_Cat1)

In [99]:
# Obtención de las palabras más utilizadas categoría 2
mcw_Cat2 = text_analysis(ShortDescESP_Cat2)

In [100]:
# Obtención de las palabras más utilizadas categoría 3
mcw_Cat3 = text_analysis(ShortDescESP_Cat3)

In [101]:
#def transform_words(serie):
#    result=[]
#    for s in serie:
#        #s[0]=s[0].encode('utf-8')
#        print(s[0])
#        result.append((s[0].encode('utf-8'),s[1]))
#        #string.replace(s[0].decode('latin1'), u'\xf1', 'ñ')
#    return result

In [102]:
#mcw_cero_donations

In [103]:
#mcw_goal_donations

#### Generación de datos para creación de gráficos

In [104]:
df_mcw_Cat1 = pd.DataFrame(mcw_Cat1)
df_mcw_Cat2 = pd.DataFrame(mcw_Cat2)
df_mcw_Cat3 = pd.DataFrame(mcw_Cat3)

In [105]:
df_mcw_Cat1.to_excel('data/ShortDescEspCat1.xlsx') 
df_mcw_Cat2.to_excel('data/ShortDescEspCat2.xlsx')
df_mcw_Cat3.to_excel('data/ShortDescEspCat3.xlsx') 

In [106]:
## Creación de base de datos para realizar gráficos de la short description en Español
#df_mcw_goal_donations=pd.DataFrame(mcw_goal_donations)
#df_mcw_goal_donations.to_excel('data/short_description_words_goal_ESP.xlsx')

In [107]:
#df_mcw_cero_donations=pd.DataFrame(mcw_cero_donations)
#df_mcw_cero_donations.to_excel('data/short_description_words_cero_ESP.xlsx')

## Obtención de long description en Español

In [108]:
# Long Description Español Categoría 1 (recaudación<35%)
LongDescESP_Cat1=data_less_than_35['Long Desc SP']
# Long Description Español Categoría 2 (35<recaudación<=70%)
LongDescESP_Cat2=data_between_36_and_70['Long Desc SP']
# Long Description Español Categoría 3 (70<recaudación)
LongDescESP_Cat3=data_grater_than_70['Long Desc SP']

#### Quitando registros sin descripción

In [109]:
LongDescESP_Cat1=LongDescESP_Cat1.dropna()
LongDescESP_Cat2=LongDescESP_Cat2.dropna()
LongDescESP_Cat3=LongDescESP_Cat3.dropna()

# Análisis de texto para Long Description

In [110]:
mcw_Long_Cat1 = text_analysis(LongDescESP_Cat1)
mcw_Long_Cat2 = text_analysis(LongDescESP_Cat2)
mcw_Long_Cat3 = text_analysis(LongDescESP_Cat3)

In [111]:
df_mcw_Long_Cat1=pd.DataFrame(mcw_Long_Cat1)
df_mcw_Long_Cat2=pd.DataFrame(mcw_Long_Cat2)
df_mcw_Long_Cat3=pd.DataFrame(mcw_Long_Cat3)

In [112]:
df_mcw_Long_Cat1.to_excel('data/LongDescEspCat1.xlsx')
df_mcw_Long_Cat2.to_excel('data/LongDescEspCat2.xlsx')
df_mcw_Long_Cat3.to_excel('data/LongDescEspCat3.xlsx')

# Descripciones en Inglés

### Obtención de short description en Inglés para 3 categorías

In [117]:
# Short Description Inglés:
#     Categoría 1 (recaudación<35%)
#     Categoría 2 (35<recaudación<=70%)
#     Categoría 3 (70<recaudación)
shortDescENG_Cat1=data_less_than_35[u'Short Desc EN']
shortDescENG_Cat2=data_between_36_and_70[u'Short Desc EN']
shortDescENG_Cat3=data_grater_than_70[u'Short Desc EN']

### Obtención de long description en Inglés para 3 categorías

In [118]:
# Long Description Inglés:
#     Categoría 1 (recaudación<35%)
#     Categoría 2 (35<recaudación<=70%)
#     Categoría 3 (70<recaudación)
longDescENG_Cat1=data_less_than_35[u'Long Desc EN']
longDescENG_Cat2=data_between_36_and_70[u'Long Desc EN']
longDescENG_Cat3=data_grater_than_70[u'Long Desc EN']

#### Quitando registros sin descripción

In [120]:
shortDescENG_Cat1=shortDescENG_Cat1.dropna()
shortDescENG_Cat2=shortDescENG_Cat2.dropna()
shortDescENG_Cat3=shortDescENG_Cat3.dropna()

longDescENG_Cat1=longDescENG_Cat1.dropna()
longDescENG_Cat2=longDescENG_Cat2.dropna()
longDescENG_Cat3=longDescENG_Cat3.dropna()

### Análisis de descripciones en Inglés

In [121]:
mcw_short_EN_Cat1 = text_analysis(shortDescENG_Cat1)
mcw_short_EN_Cat2 = text_analysis(shortDescENG_Cat2)
mcw_short_EN_Cat3 = text_analysis(shortDescENG_Cat3)

mcw_long_EN_Cat1 = text_analysis(longDescENG_Cat1)
mcw_long_EN_Cat2 = text_analysis(longDescENG_Cat2)
mcw_long_EN_Cat3 = text_analysis(longDescENG_Cat3)

In [122]:
# pasando los comentarios a DataFrame
# para incorporarlos a datos para graficar
df_mcw_short_EN_Cat1=pd.DataFrame(mcw_short_EN_Cat1)
df_mcw_short_EN_Cat2=pd.DataFrame(mcw_short_EN_Cat2)
df_mcw_short_EN_Cat3=pd.DataFrame(mcw_short_EN_Cat3)

df_mcw_long_EN_Cat1=pd.DataFrame(mcw_long_EN_Cat1)
df_mcw_long_EN_Cat2=pd.DataFrame(mcw_long_EN_Cat2)
df_mcw_long_EN_Cat3=pd.DataFrame(mcw_long_EN_Cat3)

In [127]:
# Creación de datos para graficar
df_mcw_short_EN_Cat1.to_excel('data/ShortDesEngCat1.xlsx')
df_mcw_short_EN_Cat2.to_excel('data/ShortDesEngCat2.xlsx')
df_mcw_short_EN_Cat3.to_excel('data/ShortDesEngCat3.xlsx')
df_mcw_long_EN_Cat1.to_excel('data/LongDesEngCat1.xlsx')
df_mcw_long_EN_Cat2.to_excel('data/LongDesEngCat2.xlsx')
df_mcw_long_EN_Cat3.to_excel('data/LongDesEngCat3.xlsx')